In [1]:
import calcbench as cb
import json
import itertools
from tqdm import tqdm, tqdm_notebook
import random
import urllib.error
import requests
import sys
from bs4 import BeautifulSoup
import backoff
import logging
from datetime import date, timedelta
logging.getLogger('backoff').addHandler(logging.StreamHandler())
logging.getLogger('backoff').setLevel(logging.INFO)

In [2]:
@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException), 8)
def get_doc_contents(doc):
    return doc.get_contents()

In [11]:
file_name = 'guidance_outlook.txt'
disclosure_name = 'Guidance/Outlook'
open(file_name, 'w').close()

found_documents = cb.document_search(entire_universe=True, 
                                     document_name=disclosure_name,  
                                     all_history=True)

for doc in tqdm_notebook(found_documents, leave=False):
    try:
        contents = get_doc_contents(doc)
    except requests.RequestException:
        missing_docs.append(doc)
    except json.JSONDecodeError:
        print(disclosure_name, year, period, doc['ticker'])
    contents = BeautifulSoup(contents, 'html.parser').text
    title = '{0} {1}-{2} filing date:{3}'.format(doc['ticker'], 
                                                 doc['fiscal_year'], 
                                                 doc['fiscal_period'], 
                                                 doc['filing_date'])
    output_str = '{0}\n\n{1}\n{2}\n'.format(title, contents, '-' * 80)
    output_bytes = output_str.encode(encoding='utf-8', errors='ignore')
    with open(file_name, 'ba') as output:
        output.write(output_bytes)

KeyboardInterrupt: 